In [2]:
pip install facenet-pytorch opencv-python torch torchvision


   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
    --------------------------------------- 0.0/1.9 MB 1.9 MB/s eta 0:00:01
   --- ------------------------------------ 0.2/1.9 MB 2.4 MB/s eta 0:00:01
   ------ --------------------------------- 0.3/1.9 MB 2.9 MB/s eta 0:00:01
   --------- ------------------------------ 0.5/1.9 MB 3.1 MB/s eta 0:00:01
   ---------------- ----------------------- 0.8/1.9 MB 4.4 MB/s eta 0:00:01
   ------------------------- -------------- 1.2/1.9 MB 5.8 MB/s eta 0:00:01
   -------------------------------------- - 1.8/1.9 MB 7.3 MB/s eta 0:00:01
   ---------------------------------------  1.9/1.9 MB 7.5 MB/s eta 0:00:01
   ---------------------------------------  1.9/1.9 MB 7.5 MB/s eta 0:00:01
   ---------------------------------------  1.9/1.9 MB 7.5 MB/s eta 0:00:01
   ---------------------------------------- 1.9/1.9 MB 5.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/198.5 MB ? eta -:--:--
   ---------------------

In [10]:
import cv2
import torch
import numpy as np
from facenet_pytorch import MTCNN, InceptionResnetV1
from PIL import Image
import torch.nn.functional as F
import os
from scipy.spatial.distance import cosine

In [4]:
data = np.load("C:/Users/khoi0/Downloads/Git/Do_An_Co_So/code/code_final/data_split/train_embeddings.npz")
features = data['features']
labels = data['labels']

In [5]:
features

array([[ 0.05182552, -0.01955772,  0.00527787, ..., -0.05091346,
         0.01172688,  0.0255842 ],
       [ 0.06401048, -0.00511122,  0.00847858, ..., -0.02113621,
         0.02251412,  0.00119076],
       [ 0.0465744 , -0.03645871, -0.00871841, ..., -0.03950331,
         0.02732633,  0.03983652],
       ...,
       [-0.00964092,  0.06481379,  0.00082132, ..., -0.07109835,
         0.04013845,  0.05097667],
       [-0.00283962,  0.05466783,  0.01388439, ..., -0.07835132,
         0.01795041,  0.0596398 ],
       [-0.00360568,  0.04811885,  0.00595553, ..., -0.07357092,
         0.04248341,  0.05154103]], dtype=float32)

In [6]:
labels

array(['Đạt', 'Đạt', 'Đạt', 'Đạt', 'Đạt', 'Đức', 'Đức', 'Đức', 'Đức',
       'Đức', 'Đức', 'Đức', 'Đức', 'Đức', 'Đức', 'Đức', 'Đức', 'Dũng',
       'Dũng', 'Dũng', 'Dũng', 'Dũng', 'Dương', 'Dương', 'Dương', 'Dương',
       'Dương', 'Dương', 'Dương', 'Dương', 'Dương', 'Dương', 'Dương',
       'Dương', 'Dương', 'Dương', 'Dương', 'Dương', 'Dương', 'Dương',
       'Dương', 'Dương', 'Dương', 'Dương', 'Dương', 'Dương', 'Dương',
       'Dương', 'Dương', 'Dương', 'Dương', 'Dương', 'Dương', 'Dương',
       'Dương', 'Dương', 'Dương', 'Dương', 'Dương', 'Dương', 'Dương',
       'Duy Khôi', 'Duy Khôi', 'Duy Khôi', 'Duy Khôi', 'Duy Khôi', 'Hà',
       'Hà', 'Hà', 'Hà', 'Hà', 'Hiếu', 'Hiếu', 'Hiếu', 'Hiếu', 'Hiếu',
       'Hiếu', 'Hiếu', 'Hiếu', 'Hiếu', 'Hiếu', 'Hiếu', 'Hiếu', 'Hiếu',
       'Hiếu', 'Hiếu', 'Hiếu', 'Hiếu', 'Hiếu', 'Hiếu', 'Hiếu', 'Hiếu',
       'Hiếu', 'Hiếu', 'Hiếu', 'Hiếu', 'Hiếu', 'Hiếu', 'Hiếu', 'Hiếu',
       'Hiếu', 'Hiếu', 'Hiếu', 'Hiếu', 'Hiếu', 'Hiếu', 'Hiếu', 'Hiếu',
 

In [7]:
# Dùng gpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
# Phát hiện khuôn mặt
mtcnn = MTCNN(keep_all=False, device=device)

In [9]:
# Mô hình FaceNet
model = InceptionResnetV1(pretrained='vggface2').eval().to(device)

In [ ]:
from scipy.spatial.distance import cosine

def recognize_face(face_embedding, threshold=0.7):
    min_dist = float('inf')
    identity = "Unknown"
    for i, emb in enumerate(features):
        dist = cosine(face_embedding, emb)
        if dist < min_dist:
            min_dist = dist
            identity = labels[i]

    similarity = 1 - min_dist  # cosine similarity: 1 = giống hệt, 0 = không liên quan

    if min_dist > threshold:
        return "Unknown", similarity
    return identity, similarity



In [ ]:
# Mở webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Phát hiện khuôn mặt
    boxes, probs = mtcnn.detect(frame)
    if boxes is not None:
        for box in boxes:
            x1, y1, x2, y2 = [int(b) for b in box]
            face = frame[y1:y2, x1:x2]
            face = cv2.resize(face, (160, 160))
            face = torch.tensor(face).permute(2, 0, 1).float().unsqueeze(0) / 255.
            face = face.to(device)

            with torch.no_grad():
                embedding = model(face).cpu().numpy().flatten()

            name, similarity = recognize_face(embedding)
            confidence = f"{similarity * 100:.2f}%"

            # Vẽ bounding box và tên + phần trăm
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"{name} ({confidence})", (x1, y1 - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (36, 255, 12), 2)

    cv2.imshow('Face Recognition', frame)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()